# Intermediate Random Forests Regression

## Install TensorFlow Decision Forests

In [1]:
!pip install tensorflow==2.9.1 -U -qq
!pip install tensorflow_decision_forests==0.2.7 -U -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tensorflow-transform 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tensorflow-io 0.21.0 requires tensorflow<2.7.0,>=2.6.0, but you have tensorflow 2.9.1 which is incompatible.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, but you have tensorflow-io-gcs-filesystem 0.27.0 which is incompatible.
flax 0.6.0 requires rich~=11.1, but you have rich 12.1.0 which is incompatible.


## Import Libraries

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import math
from tensorflow import keras

import matplotlib.pyplot as plt

print("TensorFlow Version: {}".format(tf.__version__))
print("TensorFlow Decision Forests: {}".format(tfdf.__version__))

rnd_seed =  42
validation_ratio = 0.1
np.random.seed(rnd_seed)
tf.random.set_seed(rnd_seed)

VALID_RATIO = validation_ratio

2022-09-16 19:30:28.933450: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-09-16 19:30:28.933492: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TensorFlow Version: 2.9.1
TensorFlow Decision Forests: 0.2.7


# Load the dataset

In [3]:
train_file_path = "../input/tabular-playground-series-aug-2021/train.csv"
train_full_data = pd.read_csv(train_file_path)
print("Full train dataset shape is {}".format(train_full_data.shape))

Full train dataset shape is (250000, 102)


In [4]:
train_full_data.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
0,0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1


The data is composed of 102 columns all of which are numerical:
* 100 feature columns named `f0, f1, ... f99`
* label column named `loss`
* An `id` column that we will drop

In [5]:
train_full_data = train_full_data.drop('id', axis=1)
features = [f'f{i}' for i in range(1, 100)]
label = 'loss'

In [6]:
train_full_data[features].isna().sum()

f1     0
f2     0
f3     0
f4     0
f5     0
      ..
f95    0
f96    0
f97    0
f98    0
f99    0
Length: 99, dtype: int64

In [7]:
def split_dataset(dataset, test_ratio=0.1):
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]

# Preprocessing

In [8]:
train_ds_pd, valid_ds_pd = split_dataset(train_full_data, test_ratio=VALID_RATIO)
print("{} samples in training and {} in validation".format(train_ds_pd.shape[0], valid_ds_pd.shape[0]))

224876 samples in training and 25124 in validation


In [9]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, 
                                                 label=label, 
                                                 task = tfdf.keras.Task.REGRESSION)
valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, 
                                                 label=label, 
                                                 task = tfdf.keras.Task.REGRESSION)

/opt/conda/lib/python3.7/site-packages/tensorflow_decision_forests/keras/core.py:2574: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
2022-09-16 19:30:41.660561: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-09-16 19:30:41.660609: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-16 19:30:41.660641: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2f19101fd603): /proc/driver/nvidia/version does not exist
2022-09-16 19:30:41.661014: I tensorflow/core/p

# GradientBoostedTreesModel Training

In [10]:
model = tfdf.keras.GradientBoostedTreesModel(task = tfdf.keras.Task.REGRESSION)

Use /tmp/tmp7ppmrbuk as temporary training directory


The next cell will take some time to get executed.

In [11]:
model.fit(train_ds, verbose=0)

[INFO kernel.cc:1176] Loading model from path /tmp/tmp7ppmrbuk/model/ with prefix 7fa70e1ccbe94560
[INFO abstract_model.cc:1248] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO kernel.cc:1022] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


# Post Training Analysis

`model.summary()` shows us the overall structure of the model

In [12]:
model.summary()

Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: REGRESSION
Label: "__LABEL"

Input Features (100):
	f0
	f1
	f10
	f11
	f12
	f13
	f14
	f15
	f16
	f17
	f18
	f19
	f2
	f20
	f21
	f22
	f23
	f24
	f25
	f26
	f27
	f28
	f29
	f3
	f30
	f31
	f32
	f33
	f34
	f35
	f36
	f37
	f38
	f39
	f4
	f40
	f41
	f42
	f43
	f44
	f45
	f46
	f47
	f48
	f49
	f5
	f50
	f51
	f52
	f53
	f54
	f55
	f56
	f57
	f58
	f59
	f6
	f60
	f61
	f62
	f63
	f64
	f65
	f66
	f67
	f68
	f69
	f7
	f70
	f71
	f72
	f73
	f74
	f75
	f76
	f77
	f78
	f79
	f8
	f80
	f81
	f82
	f83
	f84
	f85
	f86
	f87
	f88
	f89
	f9
	f90
	f91
	f92
	f93
	f94
	f95
	f96
	f97
	f98
	f99

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1. "__LABEL"  4.858066 ################
    2.     "f20"  4.837570 #####

In [13]:
inspector = model.make_inspector()
print("Model contains {} trees".format(inspector.num_trees()))

Model contains 178 trees


In [14]:
inspector.features()

["f0" (1; #0),
 "f1" (1; #1),
 "f10" (1; #2),
 "f11" (1; #3),
 "f12" (1; #4),
 "f13" (1; #5),
 "f14" (1; #6),
 "f15" (1; #7),
 "f16" (1; #8),
 "f17" (1; #9),
 "f18" (1; #10),
 "f19" (1; #11),
 "f2" (1; #12),
 "f20" (1; #13),
 "f21" (1; #14),
 "f22" (1; #15),
 "f23" (1; #16),
 "f24" (1; #17),
 "f25" (1; #18),
 "f26" (1; #19),
 "f27" (1; #20),
 "f28" (1; #21),
 "f29" (1; #22),
 "f3" (1; #23),
 "f30" (1; #24),
 "f31" (1; #25),
 "f32" (1; #26),
 "f33" (1; #27),
 "f34" (1; #28),
 "f35" (1; #29),
 "f36" (1; #30),
 "f37" (1; #31),
 "f38" (1; #32),
 "f39" (1; #33),
 "f4" (1; #34),
 "f40" (1; #35),
 "f41" (1; #36),
 "f42" (1; #37),
 "f43" (1; #38),
 "f44" (1; #39),
 "f45" (1; #40),
 "f46" (1; #41),
 "f47" (1; #42),
 "f48" (1; #43),
 "f49" (1; #44),
 "f5" (1; #45),
 "f50" (1; #46),
 "f51" (1; #47),
 "f52" (1; #48),
 "f53" (1; #49),
 "f54" (1; #50),
 "f55" (1; #51),
 "f56" (1; #52),
 "f57" (1; #53),
 "f58" (1; #54),
 "f59" (1; #55),
 "f6" (1; #56),
 "f60" (1; #57),
 "f61" (1; #58),
 "f62" (1; #59

In [15]:
inspector.variable_importances()

{'SUM_SCORE': [("f81" (1; #80), 189927.5830426521),
  ("f52" (1; #48), 162042.3216093867),
  ("f25" (1; #18), 130741.97373789968),
  ("f77" (1; #75), 114620.93633825658),
  ("f69" (1; #66), 112926.31178098707),
  ("f50" (1; #46), 104211.33111748495),
  ("f3" (1; #23), 100200.5467056674),
  ("f96" (1; #96), 98815.58570448821),
  ("f13" (1; #5), 96692.05350960791),
  ("f84" (1; #83), 88828.51105879107),
  ("f70" (1; #68), 86197.87255731504),
  ("f48" (1; #43), 85784.11569621717),
  ("f28" (1; #21), 83196.46958618308),
  ("f74" (1; #72), 82317.67313048337),
  ("f66" (1; #63), 81388.81090194592),
  ("f41" (1; #36), 80851.31664697081),
  ("f65" (1; #62), 80440.25100117456),
  ("f93" (1; #93), 77738.87079761503),
  ("f64" (1; #61), 75362.70243567321),
  ("f46" (1; #41), 75071.73715293547),
  ("f51" (1; #47), 74757.89835909917),
  ("f31" (1; #25), 69208.52136298362),
  ("f80" (1; #79), 68002.5317304912),
  ("f57" (1; #53), 66843.6995032914),
  ("f58" (1; #54), 66434.9504761449),
  ("f73" (1; 

In [16]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0, max_depth=3)

# Test Set Prediction

Although the [Tabular Playground Series - Aug 2021](https://www.kaggle.com/c/tabular-playground-series-aug-2021) has officially ended, you can still perform a Late Submission and compare the results with others on the leaderboard, especially if you want to tweak the notebook and apply some changes.

In [17]:
test_file_path = "../input/tabular-playground-series-aug-2021/test.csv"
test_data = pd.read_csv(test_file_path)
ids = test_data.pop('id')

In [18]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data)

In [19]:
preds = model.predict(test_ds)

150/150 [==============================] - 12s 75ms/step


In [20]:
output = pd.DataFrame({'id': ids,
                       'loss': preds.squeeze()})

output.head()

,id,loss
0,250000,7.279377
1,250001,4.962571
2,250002,6.888628
3,250003,7.122888
4,250004,7.579335


In [21]:
sample_submission_df = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
sample_submission_df['loss'] = model.predict(test_ds)
sample_submission_df.to_csv('/kaggle/working/submission.csv', index=False)
sample_submission_df.head()

150/150 [==============================] - 11s 75ms/step


,id,loss
0,250000,7.279377
1,250001,4.962571
2,250002,6.888628
3,250003,7.122888
4,250004,7.579335


# References


*   [KerasTuner + TF Decision Forest](https://www.kaggle.com/ekaterinadranitsyna/kerastuner-tf-decision-forest?linkId=133421702) by [Ekaterina Dranitsyna](https://www.kaggle.com/ekaterinadranitsyna)
*   [TensorFlow Decision Forests tutorials](https://www.tensorflow.org/decision_forests/tutorials) which are a set of 3 very interesting (beginner, intermediate and advanced levels) tutorials.
*   The [TensorFlow Forum](https://discuss.tensorflow.org/) where one can get in touch with the TensorFlow community. Check it out if you haven't yet.

